# Resumen de las sentencias empleadas en el curso.


### Instalar librerías recomendadas

In [ ]:
!pip install -q openai # Instala la librería de OpenAI
!pip install langchain==0.1.20 -Uq # Instala la librería de LangChain
!pip install langchain-openai==0.1.3 -Uq # Instala la librería de LangChain para OpenAI
!pip install langchain-groq==0.1.2 -Uq # Instala la librería de LangChain para Groq
!pip install langchain-anthropic==0.1.12 -Uq # Instala la librería de LangChain para Anthropic

!pip install pypdf -Uq # Instala la librería de PyPDF
!pip install faiss-cpu -Uq # Instala la librería de Faiss como BD vectorial, se puede cambiar por faiss-gpu
!pip install beautifulsoup4 # Instala la librería para hacer scrapping

## Primeras pruebas

#### Ejemplo probando directamente la API de OpenAI

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY) # Importar la API Key de OpenAI

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Eres un cocinero experto en recetas orientales"},
    {"role": "user", "content": "Por favor explícame sobre la importancia de la disciplina a la hora de cocinar"}
  ]
)

print(completion.choices[0].message.content)

#### Realizando una prompt simple con LangChain a OpenAI

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", api_key=openai_api_key) # Importar la API Key de OpenAI
prompt = "Hola, ¿cómo estás? ¿Con qué modelo de lenguaje tengo el gusto de estar hablando? ¿Exactamente qué modelo sos?"
respuesta = llm.invoke(prompt)
print(respuesta)

#### Realizando una prompt simple con LangChain a Groq

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192", temperature=0, api_key=groq_api_key) # Importar la API Key de Groq
prompt = "Hola, ¿cómo estás? ¿Con qué modelo de lenguaje tengo el gusto de estar hablando? ¿Exactamente qué modelo sos?"
respuesta = llm.invoke(prompt)
print(respuesta)

#### Realizando una prompt simple con LangChain a Anthropic

In [ ]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model='claude-3-opus-20240229', temperature=0, api_key=anthropic_api_key) # Importar la API Key de Anthropic
prompt = "Hola, ¿cómo estás? ¿Con qué modelo de lenguaje tengo el gusto de estar hablando? ¿Exactamente qué modelo sos?"
respuesta = llm.invoke(prompt)
print(respuesta)

## Manejo de cadenas

#### Ejemplo de cadena simple con LangChain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

texto="¡Hola mundo!"
template = "Traduce del español al {idioma} el siguiente texto: '{texto}'"
prompt_template = PromptTemplate(
    input_variables=["idioma","texto"], template=template
)

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", api_key=openai_api_key) # Importar la API key de OpenAI

chain = LLMChain(llm=llm, prompt=prompt_template)

respuesta = chain.invoke(input={"idioma":"italiano","texto":texto})
print(respuesta["text"])

#### Ejemplo de cadena secuencial con LangChain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.chains import LLMChain, SimpleSequentialChain

"""
En el ejemplo se utilizan dos modelos distintos para cada cadena, la salida de la primera es la entrada de
de la segunda. Además de llamo de una forma mas sencilla a PromptTemplate con from_template
"""

template1 = "Genérame el código de una función en Python que realice lo siguiente: '{accion}'"
prompt_template1 = PromptTemplate.from_template(template1)
llm1 = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", api_key=openai_api_key) # Importar la API key de OpenAI
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

template2 = "Dada una {función}, por verifica si se encuentra correctamente codificada, y explicame lo que hace"
prompt_template2 = PromptTemplate.from_template(template2)
llm2 = ChatGroq(model="llama3-70b-8192", temperature=0, api_key=groq_api_key) # Importar la API key de Groq
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

cadena_secuencial = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
salida = cadena_secuencial.invoke("Calcula todos los números primos existentes hasta 100")
print(salida['output'])


## Manejo de memoria

#### Conversation Buffer memory

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

"""
Es importante de tener en cuenta que para manejar memoria debemos invocar una cadena especial. Para el ejemplo
es ConversationChain
"""

llm = ChatAnthropic(model='claude-3-opus-20240229', temperature=0, api_key=anthropic_api_key) # Importar la API key de Anthropic

buffer_memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory = buffer_memory,
    verbose=True
)

conversation.invoke(input="Hola, mi nombre es Juan y me gusta ver partidos de futbol. La semana pasada vi 6 partidos completos.")
conversation.invoke(input="En los dos primeros días de esta semana ya he visto 4 partidos. ¿Cómo Crees que voy respecto de la cantidad de partidos que vi la semana pasada?")
conversation.invoke(input="¿Recuerdas mi nombre?")

print(buffer_memory.buffer)

summary_memory.load_memory_variables({})

#### Conversation buffer window memory

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

"""
Es importante de tener en cuenta que para manejar memoria debemos invocar una cadena especial. Para el ejemplo
es ConversationChain
"""

llm = ChatAnthropic(model='claude-3-opus-20240229', temperature=0, api_key=anthropic_api_key) # Importar la API key de Anthropic

window_memory = ConversationBufferWindowMemory(k=2) # K indica la cantida de mensajes que se van a recordar

conversation = ConversationChain(
    llm=llm,
    memory = window_memory,
    verbose=True
)

conversation.invoke(input="Hola, mi nombre es Juan. ¿Te gusta el cine?")
conversation.invoke(input="¿Cuáles son tus películas favoritas?")
conversation.invoke(input="¿Te gusta el cine argentino? ¿Cuáles son tus películas favoritas?")
conversation.invoke(input="¿Recuerdas mi nombre?")

print(window_memory.buffer)

summary_memory.load_memory_variables({})

#### Conversation token buffer memory

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain.memory import ConversationTokenBufferMemory
from langchain.chains import ConversationChain

"""
Es importante de tener en cuenta que para manejar memoria debemos invocar una cadena especial. Para el ejemplo
es ConversationChain
"""

llm = ChatAnthropic(model='claude-3-opus-20240229', temperature=0, api_key=anthropic_api_key) # Importar la API key de Anthropic

token_memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=500
) # max_token_limit indica la cantidad de tokens que se van a recordar

conversation = ConversationChain(
    llm=llm,
    memory = token_memory,
    verbose=True
)

conversation.invoke(input="Hola, mi nombre es Juan. ¿Te gusta el cine?")
conversation.invoke(input="¿Cuáles son tus películas favoritas?")
conversation.invoke(input="¿Te gusta el cine argentino? ¿Cuáles son tus películas favoritas?")
conversation.invoke(input="¿Recuerdas mi nombre?")

print(token_memory.buffer)

summary_memory.load_memory_variables({})

#### Conversation summary memory

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

"""
Es importante de tener en cuenta que para manejar memoria debemos invocar una cadena especial. Para el ejemplo
es ConversationChain
"""

llm = ChatAnthropic(model='claude-3-opus-20240229', temperature=0, api_key=anthropic_api_key) # Importar la API key de Anthropic

summary_memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100
) # max_token_limit indica la cantidad de tokens que se van a resumir

conversation = ConversationChain(
    llm=llm,
    memory = summary_memory,
    verbose=True
)

conversation.invoke(input="Hola, mi nombre es Juan. ¿Te gusta el cine?")
conversation.invoke(input="¿Cuáles son tus películas favoritas?")
conversation.invoke(input="¿Te gusta el cine argentino? ¿Cuáles son tus películas favoritas?")
conversation.invoke(input="¿Recuerdas mi nombre?")

print(summary_memory.buffer)

summary_memory.load_memory_variables({})

## RAG

#### Ejemplo con lectura de PDF

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA

"""
Es importante de tener en cuenta que para manejar memoria debemos invocar una cadena especial. Para el ejemplo
es RetrievalQA
"""

# Cargo el documento PDF
loader = PyPDFLoader("/content/Anexo Curso_ Desarrollo de Aplicaciones LLM con LangChain, LlamaIndex y OpenAI.pdf")
documento = loader.load()
# print(len(documento))

# Divido el texto en fragmentos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)
document_chunks = text_splitter.split_documents(documento)
#print(f"Ahora tenemos {len(document_chunks)} partes.")

# Creo la BD de embeddings
embeddings_model = OpenAIEmbeddings(api_key=openai_api_key) # Importar la API key de OpenAI
stored_embeddings = FAISS.from_documents(document_chunks, embeddings_model)

# Defino el modelo LLM
llm = OpenAI(api_key=openai_api_key) # Importar la API key de OpenAI

# Creo la cadena de consulta
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

question = """
De qué trata el documento
"""
QA_chain.invoke(question)

question2 = """
A quién va dirigido el curso?
"""
QA_chain.invoke(question2)

question3 = """
Cuánto cuesta el curso?
"""
QA_chain.invoke(question3)

#### Ejemplo para hacer web scrapping

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA

"""
Es importante de tener en cuenta que para manejar memoria debemos invocar una cadena especial. Para el ejemplo
es RetrievalQA
"""

# Carga el loader de scaneo de sitios webs
loader = WebBaseLoader("https://www.infobae.com/")
datos_web = loader.load()
#print(datos_web)

# Divido el texto en fragmentos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)
datos_web_chunks = text_splitter.split_documents(datos_web)
#print(f"Ahora tenemos {len(datos_web_chunks)} partes.")

# Creo la BD de embeddings
embeddings_model = OpenAIEmbeddings(api_key=openai_api_key) # Importar la API key de OpenAI
stored_embeddings = FAISS.from_documents(datos_web_chunks, embeddings_model)

# Defino el modelo LLM
llm = OpenAI(api_key=openai_api_key) # Importar la API key de OpenAI

# Creo la cadena de consulta
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

pregunta1 = """
De qué trata la web?
"""
respuesta1 = QA_chain.invoke(pregunta1)
print(respuesta1['result'])

pregunta2 = """
Qué noticias de deportes hay??
"""
respuesta2 = QA_chain.invoke(pregunta2)
print(respuesta2['result'])